<a href="https://colab.research.google.com/github/kurtyangch/Machine_Learning_from0930/blob/master/1005_Poem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 詩詞自動分類, 擷取csv並存檔
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/ml0930/raw/master/poem_train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/ml0930/raw/master/poem_test.csv"
urlretrieve(url, "test.csv")

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding = "utf-8") # 原本就是表格 用read
train_df

In [ ]:
import pandas as pd
test_df = pd.read_csv("test.csv", encoding = "utf-8")
test_df # 測試資料 

In [ ]:
# 處理資料
# Series.replace({"李白": 0}) # 把作者那行 用數字取代
# unique(拿出來的是list)/value_counts(數有幾個)
# 作者轉換成0, 1 , 2 再進行分析
writers = train_df["作者"].unique() # 拿出一個list
# result = {}
# for i, w in enumerate(writers):    # enmuerate 取出來是tuple 座號加上內容
#   result[w] = i
# result

# 上面的寫法可用下面一行代替
trans = {w:i for i, w in enumerate("writer")}
trans_r = {i:w for i, w in enumerate("writer")} # 準備一個反向字典 分析完轉換回來文字("作者")

# 進行代替
y_train = train_df["作者"].replace(trans)
y_test = test_df["作者"].replace(trans)

In [ ]:
# 內容轉換 算機率--> 要先分詞
# 標點符號, \r, \n要去掉(\r\n 換行)
# 先作其中一首
import jieba
p = train_df["內容"][0]
" ".join(jieba.cut(p))

In [ ]:
# apply(函式名字) ()-->執行
# b = int
# b(5.5)
def poemcut(p):
  return " ".join(jieba.cut(p))

x_train = train_df["內容"].apply(poemcut) # poemcut後面不用加() 把函式丟到apply()即可
x_test = test_df["內容"].apply(poemcut)

In [ ]:
# 數有幾個 CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
# fit 算出欄位
# fit_transform()的作用就是先訓練，找到轉換資料的規則，然後根據找到的規則轉換資料,tranform()的作用是根據找到的規則對資料進行轉換
x_train_count = vec.fit_transform(x_train) # 找出分類並數出數量
x_test_count = vec.transform(x_test) # 測試資料絕對不fit 因為已經固定好欄位,根據訓練出已經fit好的欄位去做預測, 訓練資料-->根據fit好的資料去transform
x_train_count
# 2731(詩的數量)x52294(詞彙數量) # sparse matrix 只儲存非0的欄位 否則資料量太大

In [ ]:
# vec.vocabulary_  #顯示分類的欄位 
# vec.vocabulary_["\r\n"] #確定標點符號, 換行 有沒有在裡面

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.1)
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [ ]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

0.8333333333333334

In [ ]:
# 看錯在哪邊 confusion matrix混淆矩陣
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre) # 參數: 正確答案 預測結果
pd.DataFrame(mat,
      index=[name + "(原本)" for name in writers],
      columns=[name + "(預測)" for name in writers])

In [ ]:
p = input("輸入一首詩:")
x_predict = vec.transform([poemcut(p)])
proba = clf.predict_proba(x_predict)[0] # 可能性機率
for w, p in zip(writers, proba): # zip 兩個list把對應的位置變成tuple
    print(w, ":", p)